# Getting started with Tethys


## Motivation

Integrated human-Earth systems models, such as GCAM, can project future water demand at a coarse, regionally-relevant scale by modeling long-term interactions between multiple sectors under a variety of scenarios, while gridded hydrology models simulate physical processes at a much finer spatial and temporal resolution. **tethys** facilitates coupling between these kinds of models by providing finer-scale water demand data while maintaining consistency with coarser-scale global dynamics.

While **tethys** is designed to integrate seamlessly with GCAM, it has the ability to downscale region-scale water demand data from other sources as well.


## Installation

As a prerequisite, you’ll need to have [Python](https://www.python.org/downloads/) installed (version 3.9 or above), and if you plan on querying a GCAM database, [Java](https://openjdk.org/) must be installed and added to your path.

**tethys** can be install from PyPi via pip:
```
pip install tethys-downscaling
```
This will automatically install the dependencies. In order to avoid package version conflicts, consider using a virtual environment.

Try importing **tethys** to confirm that installation was successful:


In [ ]:
import tethys

tethys.__version__  # should print the version number


In [ ]:
# load additional packages needed for this tutorial
# all are included with the tethys install
from pathlib import Path
from dask.distributed import Client
from matplotlib import colors
from matplotlib import pyplot as plt


## Example Data

Example data and configurations can be downloaded from Zenodo [here](https://zenodo.org/records/7569652), or by using the following:

```python
tethys.get_example_data()
```

The download decompresses to about 4.5GB. By default, it will make a directory called `example` at the root of the tethys pacakge, but you can specify another path.

Some systems do not do well unpacking data that size without first having the zipped archived downloaded.  If you would rather have the code download the zipped archive first, then unpack it, then clean up you can run:

```python
tethys.get_example_data(download_first=True)
```


## Running **tethys**

With the example data downloaded, a simple configuration can be run by the following:


In [ ]:
# assuming you downloaded to the default location
config_file = tethys.default_download_dir + '/example/config_example.yml'


Let's take a look at the config file:

In [ ]:
with open(config_file) as cfg:
    print(cfg.read())


In [ ]:
# run tethys using the config file
result = tethys.run_model(config_file)


In [ ]:
# note that the configuration opions used to conduct the run
# will be present in the `result` variable

result.years


In [ ]:
result.csv

## Plotting

**tethys** makes use of the [Xarray](https://docs.xarray.dev/en/stable/index.html) package, which provides convenient plotting functionality.  


#### View the output of Tethys for the Municipal sector for year 2020

In [ ]:
# higher dpi in order to see resolution
plt.figure(figsize=(10, 6), dpi=300)

# powernorm the color palette in order to see more detail at the low end
result.outputs.Municipal.sel(year=2020).plot(
    norm=colors.PowerNorm(0.25), 
    cmap='viridis_r',
)

plt.title("Sector: Municipal, Year: 2020")
plt.show()


#### Interactively view the input data before downscaling for the Municipal sector for year 2020

In [ ]:
from tethys import InputViewer

# instantiate input viewer
gridded_inputs = InputViewer(
    result,
    sector="Municipal",
    year=2020,
)


The data can now be plotted...

In [ ]:
gridded_inputs.plot()

Plotting options can be viewed by calling `help` on the function:

In [ ]:
help(gridded_inputs.plot)


The underlying data can also be exported to a raster file using the `.to_raster()` method:

```python
gridded_inputs.to_raster("<your raster path>")
```

## Dashboard 

**tethys** uses [Dask](https://docs.dask.org/en/stable/) for parallelization and to lazily compute results. You can launch the dask distributed client in order to view dashboard and monitor the progress of large workflows.

<div class="alert alert-block alert-info"> <b>NOTE:</b> Viewing the dashboard requires a few other dependencies not automatically installed by <b>tethys</b>. </div>


In [ ]:
# this configuration may need to be different depending on your machine
client = Client(
    threads_per_worker=8, 
    n_workers=1, 
    processes=False, 
    memory_limit='8GB'
)

# link to view the dask dashboard in your browser, probably localhost:8787
print(f"Click link to open dashboard: {client.dashboard_link}")

# run tethys AFTER launching the client
config_file = tethys.default_download_dir + '/example/config_demeter.yml'
result = tethys.run_model(config_file)

# this configuration does not write outputs to a file,
# so plots are lazily computed when requested
result.outputs.Wheat.sel(year=2030).plot(
    norm=colors.PowerNorm(0.25), 
    cmap='viridis_r'
)

plt.show()


## Additional

Check out the full user guid on our [documentation](https://jgcri.github.io/tethys/user_guide.html#) website!